In [ ]:
# Install packages
!apt install git curl -y
%pip install ftfy regex requests tqdm jupyterlab ipywidgets

# Clone repository
!git clone --recursive https://github.com/crowsonkb/v-diffusion-pytorch

# Download diffusion model
!mkdir v-diffusion-pytorch/checkpoints
!curl -L https://v-diffusion.s3.us-west-2.amazonaws.com/cc12m_1_cfg.pth > v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth

In [ ]:
%load_ext autoreload
%autoreload 2

from diffusion_model import DiffusionModel

In [ ]:
def build_params(key, type, custom={}):
    return {
        **params['types']['default'],
        **(params['types'].get(type) or {}),
        **params['prompts']['default'],
        **params['prompts'][key],
        **(params['prompts'][key].get(type) or {}),
        **custom,
    }

def run(pipeline):
    init = None
    for params in pipeline:
        print(params)
        init = DiffusionModel(params, init).run()

In [ ]:
defaults = {
    'weight': 5,
    'n_images': 1,
    'max_timestep': 1,
    'eta': 1,
    'seed': 0,
    'save': False,
    'use_init': False,
    'reverse': False,
    'size': (256, 256),
    'display_every': 10,
    'steps': 100,
    'scale': None,
}

run([
    {
        **defaults,
        'prompt': 'desert palm oasis photo',
    },    
    {
        **defaults,
        'reverse': True,
        'use_init': True,
        'max_timestep': 0.90,
        'scale': 2,
    },
    {
        **defaults,
        'prompt': 'desert palm oasis photo',
        'use_init': True,
        'max_timestep': 0.90,
    },
])